## 3 ways of loading the model:

In [48]:
from tensorflow import saved_model
from tensorflow.keras import Sequential
import tensorflow_hub as hub
import numpy as np
from PIL import Image
from tensorflow import cast, convert_to_tensor, dtypes
import pandas as pd

In [20]:
eurosat_url = 'https://tfhub.dev/google/remote_sensing/eurosat-resnet50/1'
eurosat_path = 'models/eurosat'
image_general_path = '../raw_data/raw_data/train-jpg/train-jpg'
input_shape = (256,256,3)

### Preprocess function

In [18]:
def image_preparation(image, library = 'keras'):
    image_path = f"{image_general_path}/{image}"
    image = Image.open(image_path)
    image_array = np.asarray(image)/255
    image_rgb = image_array[:,:,:3]
    image_ready = np.expand_dims(image_rgb, axis = 0)
    
    if library == 'keras':
        return image_ready
    else:
        image_tensor = convert_to_tensor(image_ready)
        image_tensor_ready = cast(image_tensor, dtypes.float32)
        return image_tensor_ready
    
    
    

In [23]:
example_tf = image_preparation('train_0.jpg', 'tensorflow')
example_keras = image_preparation('train_0.jpg', 'keras')

### Keras model:

In [21]:
model_keras = Sequential([
    hub.KerasLayer(eurosat_url, input_shape = input_shape)
])

In [24]:
model_keras.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23566282  
                                                                 
Total params: 23,566,282
Trainable params: 0
Non-trainable params: 23,566,282
_________________________________________________________________


In [29]:
prediction_keras = model_keras.predict(example_keras)
prediction_keras.shape #This way loads everything but the 10 units prediction layer, I guess for fine tuning?

1/1 [==============================] - 0s 144ms/step


(1, 2048)

### As TensorFlow model:

In [39]:
tf_model_local = saved_model.load(eurosat_path, tags = 'train') #the tag argument seems to be required 
tf_model_url = hub.load(eurosat_url)

you can use both in the same way to get predictions, which looks the same:


In [40]:
prediction_tf_local = tf_model_local.signatures['default'](example_tf)
prediction_tf_url = tf_model_url.signatures['default'](example_tf)

In [41]:
prediction_tf_local

{'logits': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-0.13254511, -2.4910147 , -1.7037553 ,  6.2905245 , -0.47602016,
         -0.6378507 ,  0.38392925, -3.8006241 ,  3.7254195 , -1.6774181 ]],
       dtype=float32)>,
 'default': <tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
 array([[0.31462836, 0.28023988, 0.31792098, ..., 0.4024513 , 0.33039102,
         0.36210364]], dtype=float32)>,
 'block1': <tf.Tensor: shape=(1, 64, 64, 256), dtype=float32, numpy=
 array([[[[-11.391829  ,  -6.866455  ,  -3.8091915 , ..., -10.987638  ,
            -0.7746881 ,   0.04535919],
          [ -9.02926   ,  -6.2174597 ,  -1.231609  , ...,  -8.725872  ,
            -1.4235663 ,  -3.1380482 ],
          [ -8.877066  ,  -6.7193985 ,  -1.0681897 , ...,  -8.529654  ,
            -1.3200866 ,  -2.0158334 ],
          ...,
          [ -9.022892  ,  -6.6509304 ,  -2.8501978 , ...,  -9.738858  ,
            -2.1201391 ,  -2.0546317 ],
          [ -8.932536  ,  -6.0339327 ,  -2.9258344 , ...,

'logits' key is the prediction:

In [42]:
prediction_tf_local['logits']

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.13254511, -2.4910147 , -1.7037553 ,  6.2905245 , -0.47602016,
        -0.6378507 ,  0.38392925, -3.8006241 ,  3.7254195 , -1.6774181 ]],
      dtype=float32)>

In [43]:
prediction_tf_url['logits']

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 9.378589 ,  6.155255 ,  6.0221224, -1.481117 , -9.341475 ,
        -8.682676 , -6.26624  , -3.6459808, -9.793684 , 16.128918 ]],
      dtype=float32)>

In [47]:
#Why are they different- same model
prediction_tf_local['logits'] == prediction_tf_url['logits']

<tf.Tensor: shape=(1, 10), dtype=bool, numpy=
array([[False, False, False, False, False, False, False, False, False,
        False]])>

### Testing
seems very hard now to find the labels, let's see if we get consistent results with what we have:

In [60]:
df = pd.read_csv('../raw_data/train_clean_labels.csv')[['image_filename', 'label']]
sample = df.sample(5_000)

In [ ]:
sample['keras_prediction']